In [3]:
import numpy as np
import pandas as pd

from pymc_marketing.mmm import MMM, GeometricAdstock, LogisticSaturation
from mmm_eval.data import generate_data
from mmm_eval.configs import PyMCConfig

# Generate data

In [4]:
data = generate_data()
data.to_csv("data.csv", index=False)

X = data.drop(columns=["revenue","quantity"])
y = data["quantity"]

# Fit a PyMC Model

In [ ]:
my_model = MMM(
    date_column="date_week" ,
    channel_columns=["channel_1","channel_2"],
    adstock=GeometricAdstock(l_max=4),
    saturation=LogisticSaturation()
)

my_model.fit(X=X, y=y, chains=4, target_accept=0.85)

# Create a config

To run the evaluation suite, we need to store the configuration used to create the original model.

To do this, we create a `PyMCConfig` object

In [6]:
# We dont need X and y - we'll get those from the input data!
fit_kwargs = { 
    "chains": 4,
    "target_accept": 0.85,
}

pymc_config = PyMCConfig(my_model, fit_kwargs=fit_kwargs, target_column="quantity")

# Check that we can use the config to fit a PyMC MMM model

In [ ]:
m2 = MMM(**pymc_config.model_config.config)
m2.fit(X=X, y=y,**pymc_config.fit_config.config)

# Save the config to JSON

In [ ]:
pymc_config.save_config(save_path="tmp/", file_name="my_config")

# And load it in a new instantiation

In [ ]:
loaded_config = PyMCConfig().load_config("tmp/my_config.json")
loaded_config.model_config.config

# Check if we can run the model from the loaded config

In [ ]:
m3 = MMM(**loaded_config.model_config.config)
m3.fit(X=X, y=y, **loaded_config.fit_config.config)